<a href="https://colab.research.google.com/github/sebasabarca/cap-comp215/blob/main/Project_1_Comp215.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Comp 215 - Project 1 - Predictive text
This program is a simple predictive text system that generates a random sentence based on the information in wikipedia from the topic provided by the user and the first two words of a sentence.

(In case you find any difficulties finding the words and the number of words that works, you can use 'Lebron_james' as teh topic, 'he won' as the first 2 words and 15 as the length of the )

Author:  Sebastian Homero Abarca  

Date:  03/03/24

In [1]:
import re
import random
import requests
from bs4 import BeautifulSoup



## The predictive dictionary
The basic idea in this predictive test is a dictionary. This dictionary has as key 2-tuple of strings of words and as values a list with the posible next word. Something like this ('I' 'love') : 'pizza', 'sushi'. With this dictionary we could lookup the next word based an the first two words and generate a sentence. Now a function is require to generate this dictionary given a string of words of any length. The function should iterate through the string and generate the word pair as keys and the next word as a value, then it should move one word and repeat the process until it registers the last word of the string.

In [ ]:
def get_predictions_dict(text):
    """Given a string of any size, returns a dictionary with 2-tuple of strings keys with a list of strings value.
    These keys are created by a slicing operator to match two words, as the keys, with the third word as the value in the dict."""

    word_dict = dict()
    words = re.sub(r'[^\w\s\']', '', text).lower().split() # Learned how to use re.sub with chat gpt

    for i in range(len(words)):
        if i == len(words) - 2:
            break
        else:
            key = (words[i], words[i+1])
            value = words[i+2]
            if key in word_dict and  word_dict[key] != [value]:
                word_dict[key].append(value)
            else:
                word_dict[key] = [value]

    return word_dict



## Unit test
Now let's run some basic unit test to check if the function is returning the desire dictionary.

In [ ]:
# Unit tests
assert get_predictions_dict("Hello my name is Sebastian. hello my brother's name is Santiago.") == {('hello', 'my'): ['name', "brother's"],
                                                                                                    ('my', 'name'): ['is'],
                                                                                                    ('name', 'is'): ['sebastian', 'santiago'],
                                                                                                    ('is', 'sebastian'): ['hello'],
                                                                                                    ('sebastian', 'hello'): ['my'],
                                                                                                    ('my', "brother's"): ['name'],
                                                                                                    ("brother's", 'name'): ['is']}

assert get_predictions_dict("I play football I play basketball I play softball but I love playing with my borther.") == {('i', 'play'): ['football', 'basketball', 'softball'],
                                                                                                                        ('play', 'football'): ['i'],
                                                                                                                        ('football', 'i'): ['play'],
                                                                                                                        ('play', 'basketball'): ['i'],
                                                                                                                        ('basketball', 'i'): ['play'],
                                                                                                                        ('play', 'softball'): ['but'],
                                                                                                                        ('softball', 'but'): ['i'],
                                                                                                                        ('but', 'i'): ['love'],
                                                                                                                        ('i', 'love'): ['playing'],
                                                                                                                        ('love', 'playing'): ['with'],
                                                                                                                        ('playing', 'with'): ['my'],
                                                                                                                        ('with', 'my'): ['borther']}


# Constructing the sentence
Now that the dictionary function is working, another function is required to create the predicted sentence. This function should take as input the length of the sentence we want to create, the first two words the user want to use, and the word dictionary the other function generated. With the first two words the function should start a list called sentence where the next words will be appended. Then, create a 2-tupple word witht the first to words to start te first lookup. The a basic for loop in range of the length of the sentence the user wants to create, will go through each two words looking for the next word chosing it randomly in case there is more than one in the list. The append this word to the sentence list and update the two_words variable.

In [ ]:
def n_predictions(n, two_words, word_dictionary):
  """Given a tupple with two words and a word dictionary, it returns n-times the next random word of a list of possible words."""

  sentence = [item for item in two_words.split()]
  two_words = tuple(two_words.lower().split())

  for i in range(n):
    next_word = random.choice(word_dictionary.get(two_words, []))
    sentence.append(next_word)
    two_words = (two_words[1], next_word)

  return sentence

## Getting the text
You should be wondering, where are we getting the text to run the get_predictions_dict() function?? I choose to make this project interactive with the user so he can use a topic of their interest to generate the dictionary of words. The easier way I found was to use Wikipedia because they have a big base data of many topics. The next funtion takes a topic that should be separated by '_' instead of spaces, insert it into the wikipedia link and then get the text using the Beatifulsoup library.  The only problem with this is that in wikipedia some words have numbers attached to them and this may generate weird words in the predicted sentence. I can't take out the numbers because they may be single values like dates or numbers that make sense to the sentence.

In [ ]:
def scrape_text_from_url(topic):
    """Scrape text data from a given topic in wikipedia."""
    try:
        url = f'https://en.wikipedia.org/wiki/{topic}'
        response = requests.get(url)

        soup = BeautifulSoup(response.content, 'html.parser') # Chat gpt helped me with the try method and beautifulsoup module
        text = soup.get_text()
        return text

    except requests.exceptions.RequestException as e:
        print("There is no info of the topic please try again")
        return None

## Main Program
Now we need to make the main program. We have to get the data from the user in the correct type and syntax. We also have to include a try loop to except any errors in case the program can't generate any sentences.

In [ ]:
def main():

  # Open a loop to try to generate the predicted words or except the index error
  while True:
    print('Welcome to the predictive text generator. You will provide two words, a topic and the number of words you wnat me to generate. Then, you will know the predicted sentence.')

    #Get the topic the user wants to use as data
    topic = input('\nPlease give only one the name of a person or one topic separated by _ (Example: Roger_Federer, Elon_Musk). This information must be available in wikipedia: ')

    #Open a loop to make sure there are no spaces
    while ' ' in topic:
      topic = input('\nPlease enter your choice separated by _ (Example: Roger_Federer, Elon_Musk): ')

    #Get the data words dictionary
    info_dict = get_predictions_dict(scrape_text_from_url(topic))

    #Ask for the two words to start the predicted sentnce and the number of words
    two_words = input('\nNow give me the first two words of the predicted sentence (Now you can enter them with normal spaces): ')

    #Open a try loop to make sure that the number provided is an interger
    while True:
      n = input('\nPlease enter an integer number: ')
      try:
          n = int(n)
          break
      except ValueError:
          print('\nInvalid input. Please enter an integer number: ')

    # Try to get the predicted words and print them with a space between them
    try:
      print('\n')
      print(' '.join(n_predictions(n, two_words, info_dict)))
      break

    except IndexError:
      print("\nIt's not possible to generate your sentence. The number of words chosen may be to many or we could match any words to the words provided. Please try again\n")

Finally we call the program and we ask the user If they want to repeat the program and start again.

In [ ]:
main()

# Ask the user if he want to repeat the program
repeat = input("\nDo you want to generate another sentce? (Y/N)").upper()
if repeat == "Y":
  print("\n")
  main()
else:
  print("\nThank you for using this program. Goodbye!")